In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, metrics
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/train',
    image_size=(512, 512),
    batch_size=64,
    label_mode='binary'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/val',
    image_size=(512, 512),
    batch_size=64,
    label_mode='binary'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/test',
    image_size=(512, 512),
    batch_size=64,
    label_mode='binary'
)

base_model = models.Sequential([
    layers.Input(shape=(512, 512, 3)),
    layers.Rescaling(1./255),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

base_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        metrics.Precision(name='precision'),
        metrics.Recall(name='recall')
    ]
)

history = base_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30
)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

precision = history.history['precision']
val_precision = history.history['val_precision']

recall = history.history['recall']
val_recall = history.history['val_recall']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(24, 6))

# Accuracy
plt.subplot(1, 4, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Val Accuracy')
plt.title('Accuracy vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Precision
plt.subplot(1, 4, 2)
plt.plot(epochs_range, precision, label='Train Precision')
plt.plot(epochs_range, val_precision, label='Val Precision')
plt.title('Precision vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()

# Recall
plt.subplot(1, 4, 3)
plt.plot(epochs_range, recall, label='Train Recall')
plt.plot(epochs_range, val_recall, label='Val Recall')
plt.title('Recall vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()

# Loss
plt.subplot(1, 4, 4)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title('Loss vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate the model on the test dataset
test_loss, test_acc, test_precision, test_recall = base_model.evaluate(test_ds)

# Print out the results
print(f"Test Accuracy: {test_acc:.3f}")
print(f"Test Precision: {test_precision:.3f}")
print(f"Test Recall: {test_recall:.3f}")
print(f"Test Loss: {test_loss:.3f}")

# Epoch Testing

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, metrics
import numpy as np
import matplotlib.pyplot as plt

# Define epoch variations
epoch_counts = [10, 20, 30, 50, 75, 100]
batch_size = 64  # Fixed batch size
dense_layers = 2  # Fixed number of dense layers

# Initialize lists to store metrics
train_accuracies, val_accuracies, test_accuracies = [], [], []
train_precisions, val_precisions, test_precisions = [], [], []
train_recalls, val_recalls, test_recalls = [], [], []
train_losses, val_losses, test_losses = [], [], []

AUTOTUNE = tf.data.AUTOTUNE

# Load datasets once since batch size is constant
train_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/train',
    image_size=(512, 512),
    batch_size=batch_size,
    label_mode='binary'
).cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/val',
    image_size=(512, 512),
    batch_size=batch_size,
    label_mode='binary'
).cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/test',
    image_size=(512, 512),
    batch_size=batch_size,
    label_mode='binary'
).cache().prefetch(buffer_size=AUTOTUNE)

for num_epochs in epoch_counts:
    print(f"\nTraining with {num_epochs} Epochs")

    # Build model
    model = models.Sequential([
        layers.Input(shape=(512, 512, 3)),
        layers.Rescaling(1./255),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten()
    ])

    # Add fixed number of Dense layers
    for _ in range(dense_layers):
        model.add(layers.Dense(128, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall')
        ]
    )

    # Train model
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=num_epochs,
    )

    # Evaluate model
    test_loss, test_acc, test_precision, test_recall = model.evaluate(test_ds)

    print(f"Epochs {num_epochs}:")
    print(f"Test Accuracy: {test_acc:.3f}")
    print(f"Test Precision: {test_precision:.3f}")
    print(f"Test Recall: {test_recall:.3f}")
    print(f"Test Loss: {test_loss:.3f}")

    # History metrics
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    precision = history.history['precision']
    val_precision = history.history['val_precision']
    recall = history.history['recall']
    val_recall = history.history['val_recall']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Store final epoch values
    train_accuracies.append(acc[-1])
    val_accuracies.append(val_acc[-1])
    test_accuracies.append(test_acc)

    train_precisions.append(precision[-1])
    val_precisions.append(val_precision[-1])
    test_precisions.append(test_precision)

    train_recalls.append(recall[-1])
    val_recalls.append(val_recall[-1])
    test_recalls.append(test_recall)

    train_losses.append(loss[-1])
    val_losses.append(val_loss[-1])
    test_losses.append(test_loss)

    # Plot metrics per epoch
    epochs_range = range(len(acc))
    plt.figure(figsize=(24, 6))

    plt.subplot(1, 4, 1)
    plt.plot(epochs_range, acc, label='Train Accuracy')
    plt.plot(epochs_range, val_acc, label='Val Accuracy')
    plt.title(f'Accuracy vs. Epochs - {num_epochs} Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 4, 2)
    plt.plot(epochs_range, precision, label='Train Precision')
    plt.plot(epochs_range, val_precision, label='Val Precision')
    plt.title(f'Precision vs. Epochs - {num_epochs} Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Precision')
    plt.legend()

    plt.subplot(1, 4, 3)
    plt.plot(epochs_range, recall, label='Train Recall')
    plt.plot(epochs_range, val_recall, label='Val Recall')
    plt.title(f'Recall vs. Epochs - {num_epochs} Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.legend()

    plt.subplot(1, 4, 4)
    plt.plot(epochs_range, loss, label='Train Loss')
    plt.plot(epochs_range, val_loss, label='Val Loss')
    plt.title(f'Loss vs. Epochs - {num_epochs} Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Plot final metrics vs number of epochs
plt.figure(figsize=(24, 6))

plt.subplot(1, 4, 1)
plt.plot(epoch_counts, train_accuracies, label='Train Accuracy', marker='o')
plt.plot(epoch_counts, val_accuracies, label='Val Accuracy', marker='o')
plt.plot(epoch_counts, test_accuracies, label='Test Accuracy', marker='o')
plt.title('Accuracy vs. Number of Epochs')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 4, 2)
plt.plot(epoch_counts, train_precisions, label='Train Precision', marker='o')
plt.plot(epoch_counts, val_precisions, label='Val Precision', marker='o')
plt.plot(epoch_counts, test_precisions, label='Test Precision', marker='o')
plt.title('Precision vs. Number of Epochs')
plt.xlabel('Number of Epochs')
plt.ylabel('Precision')
plt.legend()

plt.subplot(1, 4, 3)
plt.plot(epoch_counts, train_recalls, label='Train Recall', marker='o')
plt.plot(epoch_counts, val_recalls, label='Val Recall', marker='o')
plt.plot(epoch_counts, test_recalls, label='Test Recall', marker='o')
plt.title('Recall vs. Number of Epochs')
plt.xlabel('Number of Epochs')
plt.ylabel('Recall')
plt.legend()

plt.subplot(1, 4, 4)
plt.plot(epoch_counts, train_losses, label='Train Loss', marker='o')
plt.plot(epoch_counts, val_losses, label='Val Loss', marker='o')
plt.plot(epoch_counts, test_losses, label='Test Loss', marker='o')
plt.title('Loss vs. Number of Epochs')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


# Convolutional Layers Testing

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, metrics
import matplotlib.pyplot as plt

# Parameters
conv_layer_counts = [2, 3, 4, 5]
epochs = 30
batch_size = 64
dense_layers = 2

# Metric containers
train_accuracies, val_accuracies, test_accuracies = [], [], []
train_precisions, val_precisions, test_precisions = [], [], []
train_recalls, val_recalls, test_recalls = [], [], []
train_losses, val_losses, test_losses = [], [], []

AUTOTUNE = tf.data.AUTOTUNE

# Load datasets once
train_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/train',
    image_size=(512, 512),
    batch_size=batch_size,
    label_mode='binary'
).cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/val',
    image_size=(512, 512),
    batch_size=batch_size,
    label_mode='binary'
).cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.image_dataset_from_directory(
    'processed_dataset/test',
    image_size=(512, 512),
    batch_size=batch_size,
    label_mode='binary'
).cache().prefetch(buffer_size=AUTOTUNE)

# Function to build a CNN with variable conv layers
def build_model(n_conv_layers):
    model = models.Sequential()
    model.add(layers.Input(shape=(512, 512, 3)))
    model.add(layers.Rescaling(1./255))

    filters = [32, 64, 128, 256, 512]  # Define a max of 5 filter sizes

    for i in range(n_conv_layers):
        model.add(layers.Conv2D(filters[i], (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Flatten())

    for _ in range(dense_layers):
        model.add(layers.Dense(128, activation='relu'))

    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall')
        ]
    )
    return model

for n_conv in conv_layer_counts:
    print(f"\nTraining with {n_conv} Convolutional Layers")

    model = build_model(n_conv)

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs
    )

    test_loss, test_acc, test_precision, test_recall = model.evaluate(test_ds)

    # History data
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    precision = history.history['precision']
    val_precision = history.history['val_precision']
    recall = history.history['recall']
    val_recall = history.history['val_recall']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    train_accuracies.append(acc[-1])
    val_accuracies.append(val_acc[-1])
    test_accuracies.append(test_acc)

    train_precisions.append(precision[-1])
    val_precisions.append(val_precision[-1])
    test_precisions.append(test_precision)

    train_recalls.append(recall[-1])
    val_recalls.append(val_recall[-1])
    test_recalls.append(test_recall)

    train_losses.append(loss[-1])
    val_losses.append(val_loss[-1])
    test_losses.append(test_loss)

    # Plot epoch-wise metrics for each model
    epochs_range = range(epochs)
    plt.figure(figsize=(24, 6))

    plt.subplot(1, 4, 1)
    plt.plot(epochs_range, acc, label='Train Accuracy')
    plt.plot(epochs_range, val_acc, label='Val Accuracy')
    plt.title(f'Accuracy - {n_conv} Conv Layers')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 4, 2)
    plt.plot(epochs_range, precision, label='Train Precision')
    plt.plot(epochs_range, val_precision, label='Val Precision')
    plt.title(f'Precision - {n_conv} Conv Layers')
    plt.xlabel('Epoch')
    plt.ylabel('Precision')
    plt.legend()

    plt.subplot(1, 4, 3)
    plt.plot(epochs_range, recall, label='Train Recall')
    plt.plot(epochs_range, val_recall, label='Val Recall')
    plt.title(f'Recall - {n_conv} Conv Layers')
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.legend()

    plt.subplot(1, 4, 4)
    plt.plot(epochs_range, loss, label='Train Loss')
    plt.plot(epochs_range, val_loss, label='Val Loss')
    plt.title(f'Loss - {n_conv} Conv Layers')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Plot final metrics across different convolution layer counts
plt.figure(figsize=(24, 6))

plt.subplot(1, 4, 1)
plt.plot(conv_layer_counts, train_accuracies, label='Train Accuracy', marker='o')
plt.plot(conv_layer_counts, val_accuracies, label='Val Accuracy', marker='o')
plt.plot(conv_layer_counts, test_accuracies, label='Test Accuracy', marker='o')
plt.title('Accuracy vs. Conv Layers')
plt.xlabel('Number of Conv Layers')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 4, 2)
plt.plot(conv_layer_counts, train_precisions, label='Train Precision', marker='o')
plt.plot(conv_layer_counts, val_precisions, label='Val Precision', marker='o')
plt.plot(conv_layer_counts, test_precisions, label='Test Precision', marker='o')
plt.title('Precision vs. Conv Layers')
plt.xlabel('Number of Conv Layers')
plt.ylabel('Precision')
plt.legend()

plt.subplot(1, 4, 3)
plt.plot(conv_layer_counts, train_recalls, label='Train Recall', marker='o')
plt.plot(conv_layer_counts, val_recalls, label='Val Recall', marker='o')
plt.plot(conv_layer_counts, test_recalls, label='Test Recall', marker='o')
plt.title('Recall vs. Conv Layers')
plt.xlabel('Number of Conv Layers')
plt.ylabel('Recall')
plt.legend()

plt.subplot(1, 4, 4)
plt.plot(conv_layer_counts, train_losses, label='Train Loss', marker='o')
plt.plot(conv_layer_counts, val_losses, label='Val Loss', marker='o')
plt.plot(conv_layer_counts, test_losses, label='Test Loss', marker='o')
plt.title('Loss vs. Conv Layers')
plt.xlabel('Number of Conv Layers')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()